In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col
spark = SparkSession.builder.appName("Jupyter").getOrCreate()

spark

df = spark.read.option("header", "true").csv("/home/iceberg/data/updated_events_lab_1.csv").withColumn("event_date", expr("DATE_TRUNC('day', event_time)"))

df.show()

+-----------+--------+-----------------+---------+-------------+--------------------+--------------------+-------------------+
|        url|referrer|   browser_family|os_family|device_family|                host|          event_time|         event_date|
+-----------+--------+-----------------+---------+-------------+--------------------+--------------------+-------------------+
|          /|    NULL|             curl|    Other|        Other| www.zachwilson.tech|2023-01-09 11:55:...|2023-01-09 00:00:00|
|          /|    NULL|             curl|    Other|        Other| www.zachwilson.tech|2023-01-09 11:55:...|2023-01-09 00:00:00|
|          /|    NULL|             curl|    Other|        Other| www.zachwilson.tech|2023-01-09 20:10:...|2023-01-09 00:00:00|
|          /|    NULL|             curl|    Other|        Other| www.zachwilson.tech|2023-01-09 20:10:...|2023-01-09 00:00:00|
|          /|    NULL|            Other|    Other|        Other|admin.zachwilson....|2023-01-10 04:43:...|2023-

In [6]:
sorted = df.repartition(10, col("event_date")) \
        .sortWithinPartitions(col("event_date"), col("host"), col("browser_family")) \
        .withColumn("event_time", col("event_time").cast("timestamp")) \

sorted.show()

+---+--------------------+--------------+---------+----------------+--------------------+--------------------+-------------------+
|url|            referrer|browser_family|os_family|   device_family|                host|          event_time|         event_date|
+---+--------------------+--------------+---------+----------------+--------------------+--------------------+-------------------+
|  /|                NULL|          NULL|     NULL|            NULL|admin.zachwilson....|2023-01-01 17:22:...|2023-01-01 00:00:00|
|  /|                NULL|          NULL|     NULL|            NULL|admin.zachwilson....|2023-01-01 17:22:...|2023-01-01 00:00:00|
|  /|                NULL|       Android|  Android|Samsung SM-N900T|admin.zachwilson....|2023-01-01 17:14:...|2023-01-01 00:00:00|
|  /|                NULL|       Android|  Android|Samsung SM-N900T|admin.zachwilson....|2023-01-01 17:14:...|2023-01-01 00:00:00|
|  /|                NULL|       Android|  Android|       ZTE BA520|admin.zachwilso

In [7]:
%%sql

CREATE DATABASE IF NOT EXISTS bootcamp

++
||
++
++

In [8]:
%%sql

DROP TABLE IF EXISTS bootcamp.events

++
||
++
++

In [9]:
%%sql

CREATE TABLE IF NOT EXISTS bootcamp.events (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    device_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg
PARTITIONED BY (years(event_date));


++
||
++
++

In [10]:
%%sql


CREATE TABLE IF NOT EXISTS bootcamp.events_sorted (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    device_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg;

++
||
++
++

In [11]:
%%sql


CREATE TABLE IF NOT EXISTS bootcamp.events_unsorted (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    device_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg;

++
||
++
++

In [12]:

start_df = df.repartition(4, col("event_date")).withColumn("event_time", col("event_time").cast("timestamp")) \
    

first_sort_df = start_df.sortWithinPartitions(col("event_date"), col("browser_family"), col("host"))

sorted = df.repartition(10, col("event_date")) \
        .sortWithinPartitions(col("event_date")) \
        .withColumn("event_time", col("event_time").cast("timestamp")) \

start_df.write.mode("overwrite").saveAsTable("bootcamp.events_unsorted")
first_sort_df.write.mode("overwrite").saveAsTable("bootcamp.events_sorted")

In [13]:
%%sql

SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'sorted' 
FROM demo.bootcamp.events_sorted.files

UNION ALL
SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'unsorted' 
FROM demo.bootcamp.events_unsorted.files





size,num_files,sorted
122256,4,sorted
128854,4,unsorted


In [14]:
%%sql
SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files FROM demo.bootcamp.events.files;

size,num_files
None,0


In [15]:
%%sql 
SELECT COUNT(1) FROM bootcamp.matches_bucketed.files

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `bootcamp`.`matches_bucketed`.`files` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 21;
'Aggregate [unresolvedalias(count(1), None)]
+- 'UnresolvedRelation [bootcamp, matches_bucketed, files], [], false
